# 신용대출 심사

* 고객사는 ## 은행입니다. 신용평가 업무를 인공지능으로 전환하고자 여러분에게 모델링을 의뢰하였습니다.
* 대출업무는
    * 은행 창구에서 신청을 받고
    * 본사의 심사부서에서는 신용평가를 통해 대출 신청에 대한 승인 여부를 결정해 왔습니다.

* 현장의 요구
    * 경쟁사의 공격적인 대출상품 판매로, 본사에서는 자사 은행의 대출 실적이 줄어들고 있는 것에 부담을 느끼고 있습니다.
    * 그런데, 자사 은행에서는 신용평가 결과의 정확성에 의문을 품고 있으며, 신용평가 기준을 완화하여 가급적 대출승인 범위를 더 확대해 주기를 요구합니다.

* 신용평가 업무를 인공지능으로 전환
    * 현장의 요구를 감안하여, 과거 사람이 하던 평가방식을 개선하고자 인공지능에 의한 예측 모델을 만들고, 정확도를 높이고자 합니다.
-----------------


## 1.환경준비

### 1) 라이브러리 로딩

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.graphics.mosaicplot import mosaic

import scipy.stats as spst

### 2) 데이터 로딩

In [2]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/credit_all.csv'
data = pd.read_csv(path)
data.loc[data['Payment'] == 4, 'Payment'] = 3
data.drop(['Telephone','ForeignWorker','Dependents'], axis = 1, inplace = True)
data.head()

,Creditability,AccountBalance,Duration,Payment,Purpose,CreditAmount,Employment,SexMarital,CurrentAddress,MostValuableAsset,Age,Apartment,CreditCount,Occupation
0,1,3,24,2,0,1249,2,4,2,1,28,2,1,3
1,1,2,9,2,0,276,3,4,4,1,22,1,1,2
2,1,1,18,3,2,1049,2,2,4,2,21,1,1,3
3,1,1,24,3,1,6419,5,2,4,4,44,3,2,4
4,1,3,12,2,2,1424,5,2,4,1,55,2,1,4


|	칼럼명	|	설명	|	 	|	값 의미	|
|	-----	|	-----	|	-----	|	-----	|
|	Creditability	|	Creditability(Target)	|	향후 신용도	|	0 : Bad, 1 : Good	|
|	AccountBalance	|	Account Balance	|	은행잔고	|	1: No account, 2 : None (No balance), 3 : Some Balance	|
|	Duration	|	Duration of Credit (month)	|	신청한 대출기간(월)	|	숫자	|
|	Payment	|	Payment Status of Previous Credit	|	과거 대출 납입 상태	|	0 : 연체, 1 : 기타신용, 2 : 완납, 3 : 정상 대출상환 중 |
|	Purpose	|	Purpose	|	신청한 대출목적	|	1 : New Car , 2 : Used Car , 3 : Furniture , 4 : TV , 5 : Appliances , 6 : Repair , 8 : Vacation , 9 :Retraining , 10 : Business , 0 : Other	|
|	CreditAmount	|	Credit Amount($)	|	신청한 대출금액	|		|
|	Employment	|	Length of current employment(Month)	|	현 직업 근무 기간	|	1: Unemployed,  2: <1 Year,  3: [1, 4),  4: [4, 7),  5: Above 7	|
|	SexMarital	|	Sex & Marital Status	|	성별 & 결혼상태	|	1: Male, Divorced, 2: Male, Single , 3: Male, Married/Widowed , 4: Female	|
|	CurrentAddress	|	Duration in Current address	|	현 거주지 거주기간	|	1: <1 Year , 2: [1, 4) , 3: [4, 7) , 4: Above 7	|
|	MostValuable	|	Most valuable available asset	|	가장 가치있는 자산	|	1: None , 2: Car , 3: Life Insurance , 4: Real Estate	|
|	Age	|	Age (years)	|	나이	|		|
|	AppartmentType	|	Type of apartment	|	주거환경	|	1: free apartment, 2: Rented, 3: Owned	|
|	NoCredits	|	No of Credits at this Bank	|	현재 총 대출 건수	|	1 : one, 2 : 2 ~ 3, 3 : 4 ~ 5, 4 : 6 ~	|
|	Occupation	|	Occupation	|	직업	|	1: Unemployed, unskilled, 2: Unskilled Permanent Resident, 3: Skilled, 4: Executive	|



* 데이터의 크기를 살펴봅시다.

In [ ]:
data.shape

## 2.개별 변수 분석하기

* 파악해야 할 부분
    * 변수의 의미
    * 분포
        * 대체로 데이터가 모여 있는 부분
        * 데이터가 희박한 부분
        * 특이한 부분
    * 비즈니스 파악하기
        * 분포로 부터 비즈니스 이해하기
    * 추가 분석하고 싶은 내용

### (1) 숫자형 변수

In [ ]:
def eda_1_n(data, var, bins = 30) :
    display(data[[var]].describe().T)
    print('-' * 70)

    plt.figure(figsize = (8,6))
    plt.subplot(2,1,1)
    sns.histplot(x=var, data = data, bins = 20, kde = True)
    plt.grid()

    plt.subplot(2,1,2)
    sns.boxplot(x=var, data = data)
    plt.grid()

    plt.tight_layout()
    plt.show()

#### 1) Duration

#### 2) CreditAmount

#### 3) Age

### (2) 범주형 변수

In [ ]:
def eda_1_c(data, var) :
    print(f'<< {var} >>')
    temp1 = data[var].value_counts()
    temp2 = data[var].value_counts()/ data.shape[0]
    temp3 = pd.DataFrame({'class':temp1.index, 'count':temp1.values, 'prop':temp2.values})
    display(temp3)
    print('-' * 70)
    sns.countplot(x = var, data = data)
    plt.grid()
    plt.show()

#### 1) Creditability(target)

#### 2) AccountBalance

#### 3) Payment

#### 4) Purpose

#### 5) Employment

#### 6) SexMarital

#### 7) CurrentAddress

#### 8) MostValuableAsset

#### 9) Apartment

#### 10) CreditCount

#### 11) Occupation

## 3.두 변수의 관계 분석하기

In [ ]:
target = 'Creditability'

### (1) 숫자형 X --> 범주형 Y

* 그래프 : histogram, densityplot
* 수치화 : 로지스틱회귀  
* 관계 평가

In [ ]:
def eda_2_nc(data, target, var) :

    plt.figure(figsize = (8,6))
    plt.subplot(2,1,1)
    sns.kdeplot(x=var, data = data, hue =target, common_norm=False)
    plt.xlim(data[var].min(), data[var].max())
    plt.grid()

    plt.subplot(2,1,2)
    sns.kdeplot(x=var, data = data, hue =target, multiple='fill')
    plt.axhline(data[target].mean(), color = 'r')
    plt.xlim(data[var].min(), data[var].max())
    plt.grid()

    plt.tight_layout()
    plt.show()

#### 1) Duration --> Creditability

#### 2) CreditAmount --> Creditability

#### 3) Age --> Creditability

### (2) 범주형 X --> 범주형 Y

* 교차표(pd.crosstab())
* 시각화( Stacked Bar, Mosaic)
* 가설검정(카이제곱검정)
* 관계 평가

In [ ]:
import scipy.stats as spst

In [ ]:
def eda_2_cc(data, target, var) :

    print(data[target].mean())

    temp1 = pd.crosstab(data[target], data[var])
    display(temp1)
    temp2 = pd.crosstab(data[target], data[var], normalize = 'columns')
    display(temp2)

    mosaic(data, [ var,target], gap = 0.01)
    plt.axhline(data[target].mean(), color = 'r')
    plt.show()

    # 카이제곱검정
    print(spst.chi2_contingency(temp1))

#### 1) AccountBalance --> Creditability

#### 2) Payment --> Creditability

#### 3) Purpose --> Creditability

#### 4) Employment --> Creditability

#### 5) SexMarital --> Creditability

#### 6) MostValuableAsset --> Creditability

#### 7) Apartment --> Creditability

#### 8) CreditCount --> Creditability

#### 9) Occupation --> Creditability

### (3) 관계 정리하기

① 강한관계

② 중간관계

③ 관계없음